In [1]:
import pandas as pd
import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output, State
import plotly.express as px
import dash_bootstrap_components as dbc
import random

In [2]:
df = pd.read_csv('survey.csv',parse_dates=['Timestamp'])

In [3]:
df.head()

,Timestamp,Age,Gender,Country,state,self_employed,family_history,treatment,work_interfere,no_employees,...,leave,mental_health_consequence,phys_health_consequence,coworkers,supervisor,mental_health_interview,phys_health_interview,mental_vs_physical,obs_consequence,comments
0,2014-08-27 11:29:31,37,Female,United States,IL,NaN,No,Yes,Often,6-25,...,Somewhat easy,No,No,Some of them,Yes,No,Maybe,Yes,No,NaN
1,2014-08-27 11:29:37,44,M,United States,IN,NaN,No,No,Rarely,More than 1000,...,Don't know,Maybe,No,No,No,No,No,Don't know,No,NaN
2,2014-08-27 11:29:44,32,Male,Canada,NaN,NaN,No,No,Rarely,6-25,...,Somewhat difficult,No,No,Yes,Yes,Yes,Yes,No,No,NaN
3,2014-08-27 11:29:46,31,Male,United Kingdom,NaN,NaN,Yes,Yes,Often,26-100,...,Somewhat difficult,Yes,Yes,Some of them,No,Maybe,Maybe,No,Yes,NaN
4,2014-08-27 11:30:22,31,Male,United States,TX,NaN,No,No,Never,100-500,...,Don't know,No,No,Some of them,Yes,Yes,Yes,Don't know,No,NaN


In [4]:
#clean 'Gender'
#Slower case all columm's elements
gender = df['Gender'].str.lower()
#print(gender)

#Select unique elements
gender = df['Gender'].unique()

#Made gender groups
male_str = ["male", "m", "male-ish", "maile", "mal", "male (cis)", "make", "male ", "man","msle", "mail", "malr","cis man", "Cis Male", "cis male"]
trans_str = ["trans-female", "something kinda male?", "queer/she/they", "non-binary","nah", "all", "enby", "fluid", "genderqueer", "androgyne", "agender", "male leaning androgynous", "guy (-ish) ^_^", "trans woman", "neuter", "female (trans)", "queer", "ostensibly male, unsure what that really means"]           
female_str = ["cis female", "f", "female", "woman",  "femake", "female ","cis-female/femme", "female (cis)", "femail"]

for (row, col) in df.iterrows():

    if str.lower(col.Gender) in male_str:
        df['Gender'].replace(to_replace=col.Gender, value='male', inplace=True)

    if str.lower(col.Gender) in female_str:
        df['Gender'].replace(to_replace=col.Gender, value='female', inplace=True)

    if str.lower(col.Gender) in trans_str:
        df['Gender'].replace(to_replace=col.Gender, value='trans', inplace=True)

#Get rid of bullshit
stk_list = ['A little about you', 'p']
df = df[~df['Gender'].isin(stk_list)]

print(df['Gender'].unique())


['female' 'male' 'trans']


# Application

## css

In [5]:

# the style arguments for the main content page.
CONTENT_STYLE={
   'margin-left': '2%',
    'margin-right': '5%',
    'padding': '20px 10px 20px 10px'
}

TEXT_STYLE={
    'textAlign': 'center',
    'color': 'white',
    'padding-top': '15px'
}

CARD_TEXT_STYLE={
    'textAlign': 'center',
    'color': 'white'
}


# nav={
#     'textAlign': 'center',
#     'color': '#191970',
#     'height':'100px'
    
# }

colors = {
    'background': '#111111',
    'text': '#7FDBFF'
}


## Cards

In [6]:
cards = dbc.Row([
    dbc.Col(
        dbc.Card(
            [
                dbc.CardBody(
                    [
                        html.H4(id='card_title_1', children=['# Respondents'], className='card-title',
                                style=CARD_TEXT_STYLE),
                        html.H5(id='card1', children=[1252], style=CARD_TEXT_STYLE),
                    ],
                )
            ],style={'backgroundColor':'#53b7e8'}
        ),
        md=3
    ),
    dbc.Col(
        dbc.Card(
            [
                dbc.CardBody(
                    [
                        html.H4('# Work in Tech Company', className='card-title', style=CARD_TEXT_STYLE),
                        html.H5(id='card2', children=[random.randint(0, 200)], style=CARD_TEXT_STYLE),
                    ]
                ),
            ],style={'backgroundColor':'#53b7e8'}

        ),
        md=3
    ),
    dbc.Col(
        dbc.Card(
            [
                dbc.CardBody(
                    [
                        html.H4('# Have Mental Issues', className='card-title', style=CARD_TEXT_STYLE),
                        html.H5(id='card3', children=[random.randint(100, 300)], style=CARD_TEXT_STYLE),
                    ]
                ),
            ],style={'backgroundColor':'#53b7e8'}

        ),
        md=3
    ),
    dbc.Col( dcc.Dropdown(   
                id='country',
                options=[ {'label':str(cont),'value':str(cont) } for cont in df['Country'].unique()],
                value=None,
                placeholder='choose a country....'
            ), md=2),
  
])

## First row

In [7]:
content_first_row = dbc.Row([    
    dbc.Col( html.Div(dcc.Graph(id='benefits_pie'),style=CONTENT_STYLE), md=4 ),
    dbc.Col( html.Div(dcc.Graph(id='leave_pie'), style=CONTENT_STYLE),md=4 ),
    dbc.Col(html.Div( dcc.Graph(id='seek_help_pie'),style=CONTENT_STYLE ),md=4 ), 
],
    
)

## Second row

In [8]:
content_second_row = dbc.Row([    
    dbc.Col(html.Div(dcc.Graph(id='MvsP_conseq_fig'),style=CONTENT_STYLE), md=6),
    dbc.Col(html.Div(dcc.Graph(id='MvsP_intrv_fig'),style=CONTENT_STYLE), md=6),
],
style={'backgroundColor': '#404040'}
)

## All content

In [9]:
content = html.Div(style={'backgroundColor': '#383838'},
    children=[

        html.Div(
            children=[
                html.H2('Mental Health in Tech Industry 💆💆‍♂️',style=TEXT_STYLE),
            ] ,style={"height":'70px','backgroundColor':'#53b7e8', 'padding-bottom': '50px'}
        ),
        

        html.Div(cards,style={"height":'190px', 'backgroundColor': '#404040','padding': '40px 10px 20px 10px'}),
        html.Div(content_first_row,style={'backgroundColor':'#404040','padding-bottom': '30px'}),
        html.Div(content_second_row),
    ],
   
)

## App

In [10]:
external_stylesheets = [
    {
        "href": "https://fonts.googleapis.com/css2?"
                "family=Lato:wght@400;700&display=swap",
        "rel": "stylesheet",
    },
]

In [11]:
app = dash.Dash('Mental Health',external_stylesheets=[dbc.themes.BOOTSTRAP,external_stylesheets])
app.layout = html.Div([content])

In [12]:
@app.callback([
    Output('card1','children'),
    Output('card2','children'),
    Output('card3','children'),
    
    Output('benefits_pie','figure'),
    Output('leave_pie','figure'),
    Output('seek_help_pie','figure'),
    
    Output('MvsP_conseq_fig','figure'),
    Output('MvsP_intrv_fig','figure'),
    ],
   
    [Input('country','value')],
    
    State('country', 'value'),
)

def update(n_clicks, country):
    if country==None:
        selected_df = df  
    else:
        selected_df=df[(df.Country==country)]
    
    respondents = len(selected_df)
    work_in_tech = len(selected_df[(selected_df['tech_company']=='Yes')])
    have_family_history = len(selected_df[(selected_df['family_history']=='Yes')])

    print(respondents, work_in_tech, have_family_history)

    benefits_pie = px.pie(selected_df,names='benefits', title='Mental Health Benefits')
    
    benefits_pie.update_layout(
    plot_bgcolor='#484848',
    paper_bgcolor='#484848',
    font_color=colors['text']
    )
     
    
    leave_pie = px.pie(selected_df,names='Leave for a mental condition', title='Leave')
    
    leave_pie.update_layout(
    plot_bgcolor='#484848',
    paper_bgcolor='#484848',
    font_color=colors['text']
    )
 
    
    seek_help_pie = px.pie(selected_df,names='seek_help', title = 'Provide Helpful Resources')
    
    seek_help_pie.update_layout(
    plot_bgcolor='#484848',
    paper_bgcolor='#484848',
    font_color=colors['text']
    )
    
    
    
    
    MvsP_conseq_fig = px.histogram(selected_df, x=['mental_health_consequence','phys_health_consequence'], barmode='group', title='Negative consequence of having Mental/Physical issue')
    
    MvsP_conseq_fig.update_layout(
    plot_bgcolor='#484848',
    paper_bgcolor='#484848',
    font_color=colors['text']
    )
    
    MvsP_intrv_fig = px.histogram(selected_df, x=['mental_health_interview','phys_health_interview'], barmode='group', title='Mentioning Mental/Physical health in an interview')

    MvsP_intrv_fig.update_layout(
    plot_bgcolor='#484848',
    paper_bgcolor='#484848',
    font_color=colors['text']
    )

    return  respondents, work_in_tech, have_family_history,\
            benefits_pie, leave_pie, seek_help_pie, \
            MvsP_conseq_fig, MvsP_intrv_fig
            

In [ ]:
app.run_server()

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app 'Mental Health' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
